# LCA of Bitcoin Mining (consequential model)

Initial set up

In [ ]:
import pandas as pd
import numpy as np
from lci_to_bw2 import *
from brightway2 import *
from matplotlib import pyplot as plt
import time
when = time.strftime("%Y%m%d")

In [ ]:
projects.set_current('BTC_LCA')

In [ ]:
projects

In [ ]:
#upload consequential ecoinvent v3.5 database (& change directory to find the file on your computer)
fpei34  = "/Users/susanne/Documents/O1a/Databases/Ecoinvent_conseq_3.5/datasets"
 
if 'ecoinvent 3.5 conseq' in databases:
     print("Database has already been imported")
else:
     ei34 = SingleOutputEcospold2Importer(fpei34, 'ecoinvent 3.5 conseq')
     ei34.apply_strategies()
     ei34.statistics()
 
ei34.write_database()

In [ ]:
bw2setup() # do this if its the first time only

In [ ]:
databases

## Data import

Now import the inventory. This has to be done for each scenario. From here all steps have to be **repeated for each of the three scenarios**. 

This means: 
1. upload sceanrio BAU
2. carry out the LCA and the Monte Carlo simulation for the BAU scenario
3. upload scenario technology
4. carry out the LCA and the Monte Carlo simulation for the technology scenario
5. upload scenario location
6. carry out the LCA and the Monte Carlo simulation for the location scenario

In [ ]:
#the .csv files should be in the same folder for easy upload
#change name of file depending on which scenario is to be run
BTC_data = pd.read_csv('Consequential_BAU.csv', header = 0, sep = ";", encoding = 'utf-8-sig') 

# clean up 
BTC_data = BTC_data.drop(['Notes'], 1)  # remove the columns not needed
BTC_data['Exchange uncertainty type'] = BTC_data['Exchange uncertainty type'].fillna(0).astype(int) 

print(BTC_data.head())
print(BTC_data.tail())  

In [ ]:
# Create a dict that can be written as database
BTC_dict = lci_to_bw2(BTC_data)

In [ ]:
# Write a bw2 database
databases
if 'BTC' in databases: del databases['BTC']
BTC = Database("BTC")
BTC.write(BTC_dict)
#[print(act) for act in BTC]

## Static LCA

Now these steps are related to carrying out the LCA. The steps are always conducted on the latest .csv inventory uploaded!

In [ ]:
acts = []
for activity in Database("BTC"):
        acts.append(activity['name'])
acts.sort()
acts

In [ ]:
#all analyzed impact categories
mylist = [('IPCC 2013', 'climate change', 'GWP 100a'),
         ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'),
         ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
         ('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
         ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
         ('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
         ('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
         ('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
         ('ReCiPe Midpoint (H)', 'photochemical oxidant formation', 'POFP'),
         ('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
         ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')]

In [ ]:
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mymethod

Here the number behind acts needs to be adapted to fit the scenario being analysed (see text below for which number corresponds to which scenario). When the output says 'Bitcoin Network' (Tera Hashes, None, None) you have chosen the correct value. 

In [ ]:
#Change number in acts depending on which scenario
# 26 for BAU
# 14 Scenario 2
# 7  Scenario 3
myact = Database('BTC').get(acts[26]) 
myact

This is the result using IPCC 2013 for 1 TH in kg CO2-eq.

In [ ]:
functional_unit = {myact: 1} 
lca = LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

Everything worked so now preparing for doing this **in loop** to all activities under analysis

In [ ]:
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

def getLCAresults(acts, mymethod):
    
    all_activities = []
    results = []
    for a in acts:
        act = Database('BTC').get(a)
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict


In [ ]:
results_BTC = []
for m in mylist:
    results_all_acts = getLCAresults(acts,m) # total impact per tech
    results_BTC.append(results_all_acts)

In [ ]:
methods_names = []
for m in mylist:
    m_name = ' '.join(m)
    methods_names.append(m_name)

In [ ]:
my_output = pd.DataFrame(results_BTC, index=methods_names)

In [ ]:
print(my_output['Bitcoin Network'])


In [ ]:
#Give a proper name to the file and export
my_output.to_csv('LCA_BAU.csv', sep = ';')

## Uncertainty analysis (Monte Carlo simulation)

In [ ]:
mc = MonteCarloLCA({Database('BTC').get('Bitcoin Network'):1000000}, mymethod) 
# important to initialize the MC simulation
next(mc)

In [ ]:
# This is the Monte Carlo simulation
mc = MonteCarloLCA({Database('BTC').get('Bitcoin Network'):1000000}, mymethod) 
mc_results = [next(mc) for x in range(1000)]

In [ ]:
from matplotlib.pylab import *
hist(mc_results, density=True)  # From matplotlib package
ylabel("Probability")
print(mean(mc_results))
print(median(mc_results))
np.exp(mean(np.log(mc_results))) # geometric mean VERY close to 120
pd.DataFrame(mc_results).describe()  # Using the pandas package

In [ ]:
#Give a proper name to the file and export
df = pd.DataFrame(mc_results)
df.to_csv('MCsimulation_BAU.csv', sep = ';')